In [1]:
from ast import Return
import numpy as np
import random


In [4]:
"""

Enunciado: Ejercicio 1. Escribe una función que determine si 
una secuencia de bits cumple los postulados de Golomb

"""
def postuladosGolomb(secuencia):

    # Comprobamos cada uno de los 3 postulados con la 
    # secuencia proporcionada

    ##############################################

    # Primer postulado: que haya el mismo número de 0s y 1s, 
    # y en caso de que tenga un número impar de elementos
    # habrá a lo mucho 1 elemento de diferencia entre ambas
    size=np.size(secuencia)
    unos=np.count_nonzero(secuencia)
    ceros=size-unos

    # Depende de si tenemos un número par o impar de elementos
    # en la cadena, tambien compruebo que no tengamos cero 1s o cero 0s
    # para evitar problemas en los siguientes postulados
    if (size%2==0 and ceros!=unos) or ceros==0 or unos==0:
        return False
    elif ceros!=unos-1 and ceros!=unos+1:
        return False

    ##############################################
    
    # Segundo postulado: teniendo en cuenta que es una secuencia
    # cíclica el número de secuencias de 0s y 1s debe ser igual, 
    # además el número de secuencias de un tamaño debe de ser la mitad
    # cuando incrementamos la longitud de la secuencia en 1 elemento

    # Antes de ver si se cumple el segundo postulado, al tratarse de 
    # de secuencias cíclicas trasladamos la secuencia hasta que el primer
    # y último elemento sean distintos
    while secuencia[0]==secuencia[-1]:
        secuencia=np.roll(secuencia,1)


    # Implementación eficiente para encontrar y contar secuencias 
    # sacada de 
    # https://stackoverflow.com/questions/58221268/count-number-of-repeated-elements-in-a-row-in-a-numpy-array

    # En primer lugar lo que hace en esta implementación es para cada elemento ver si es igual
    # a su siguiente, obteniendo un array con [ True, ..., ¿n!=n+1?, ..., True], con esto pretendemos
    # tener los indices en los que cambian los elementos
    lista = np.r_[True,secuencia[:-1]!=secuencia[1:],True]

    # Aqui se calcula el número de elementos de cada secuencia, mediante la 
    # diferencia de la lista de los elementos que son True(elementos que son distintos de su siguiente)
    numElementos = np.diff(np.flatnonzero(lista))

    # Obtenemos a que elemento corresponde cada secuencia, tomando
    # los elementos de la secuencia para los que la lista calculada
    # es True
    elemento = secuencia[lista[:-1]]

    # Contamos rachas 

    # Array en el para la columna i tenemos las rachas de 
    # tamaño i+1 tanto para 0 y 1 en los indices 0 y 1
    rachas=np.zeros((2,np.max(numElementos)),dtype=np.int64)

    # Rellenamos el array a partir de la secuencia
    for i in np.arange(np.size(numElementos)):
        rachas[elemento[i]][numElementos[i]-1]+=1

    # Ahora comprobamos que se cumple el segundo postulado
    # a partir de lo calculado
    
    suma=rachas[:][0]+rachas[:][1]
    
    # No puede haber más de 2 tamaños de rachas con solo
    # 1 racha, en cuyo caso no se cumple el postulado
    if np.size(suma[np.where(suma==1)])>2:
        return False

    # No puede haber mas rachas de tamaño n que de n-1, 
    # La suma de los elementos de la diferencia del vector suma
    # que son mayores que 0 debe ser 0
    if np.sum(np.diff(suma)>0)!=0:
        return False

    # Compruebo que haya el mismo número de cadenas longitud n
    # para 0s y 1s (cuando no hay 1 sola cadena), que se cumpla
    # la mitad de cadenas etc
    for i in np.arange(np.size(suma)):
        if i==0:
            if suma[i]!=1 and rachas[0][0]!=rachas[1][0]:
                return False
        else:
            if suma[i]!=1 and ( rachas[0][0]!=rachas[1][0] or suma[i]!=suma[i-1]/2):
                return False
            if suma[i]==1 and suma[i]!=suma[i-1]/2 and i!=np.size(suma)-1:
                return False

    ##############################################

    # Tercer postulado: Se cumple si para todos los desplazamientos
    # de la secuencia el número de elementos con respecto a la secuencia
    # original se mantiene constante

    secuenciaDesp=np.roll(secuencia,1)
    desp=np.sum(secuenciaDesp!=secuencia)
    while(not np.array_equal(np.roll(secuenciaDesp,1), secuencia)):
        secuenciaDesp=np.roll(secuenciaDesp,1)
        if desp!=np.sum(secuenciaDesp!=secuencia):
            return False
        desp=np.sum(secuenciaDesp!=secuencia)

    # Si se cumplen los 3 postulados devolvemos true
    return True



In [7]:
postuladosGolomb(np.array([0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]))

[[2 1 1 0]
 [2 1 0 1]]


True

In [8]:
postuladosGolomb(np.array([0,0,0,0,1,0,0,1,0,1,1,0,0,1,1,1,1,1,0,0,0,1,1,0,1,1,1,0,1,0,1]))

[[4 2 1 1 0]
 [4 2 1 0 1]]


True

In [9]:
postuladosGolomb(np.array([0,0,1,0,0,0,1,1,1,0,1]))

[[1 1 1]
 [2 0 1]]


False

In [10]:
postuladosGolomb(np.array([0,0,0,0,0,1,0,0,0,1,1,0,1]))

False

In [11]:
# Función auxiliar para contar los bits de un 
# entero
def sumaBitsEntero(n):
    suma=0
    while(n):
        suma+=n&1
        n=n>>1
    return suma%2


"""
    Implementar LFSR.
    La funcion recibe los coeficientes del polinomio de
    conexión, la semilla o estado inicial del LFSR y la 
    longitud de la secuencia de salida.

    ##### FORMATO DE LA ENTRADA #####
        nSalida : Entero
        coeficientes : String de bits tal que "1001"
            de mayor a menor exponente, en este caso x4 + x2 + 1
            no tengo en cuenta el término independiente
        semilla : String de bits "0110"
"""
def lfsr(nSalida, coeficientes, semilla):

    # Al tener un tamaño de salida requerido, nos da 
    # igual la periodicidad de la salida, además el número
    # de stages o L va a ser el número de elementos de la
    # semilla
    lenght=len(semilla) # Guardo el tamaño de la semilla para tener
            # constancia de los 0's a la izquierda
    actual=int(semilla,2)
    coeficientes=int(coeficientes,2)
    salida=np.empty(nSalida)

    # Para el tamaño de la salida realizamos lo siguiente
    for i in np.arange(nSalida):
        # Añadimos a la salida el último elemento del 
        # estado actual( si el valor actual no es par, acaba en 1
        # , en otro caso 0)
        salida[i]=int(actual%2!=0)

        # Antes de hacer el shift, calculamos el 
        # último valor
        calculo=coeficientes & actual

        # Hacemos un shift de 1 bit
        actual=actual>>1

        # Añadimos un 0 o un 1 segundo corresponda a partir del 
        # polinomio de conexión, para ello hago un AND entre el polinomio
        # de conexión y el estado actual.
        actual=(sumaBitsEntero(calculo)<<lenght-1) | actual

    return salida



In [12]:
lfsr(10,"11","11")

array([1., 1., 0., 1., 1., 0., 1., 1., 0., 1.])

# Vaya manía con usar matrices de numpy ... y encima la salida no son ni enteros

In [13]:
def nlfsr(semilla, k):

    lenght=len(semilla)
    actual=int(semilla,2)
    salida=np.empty(k)

    for i in np.arange(k):
        salida[i]=int(actual%2!=0)
        x, y, z, t=actual & (1<<3), actual & (1<<2), actual & (1<<1), actual & 1 
        calculo=((x & y) | (1 ^ z)) ^ t
        actual=actual>>1
        
        # Función no lineal
        actual=(sumaBitsEntero(calculo)<<lenght-1) | actual

    return salida


# No admite el polinomio o el circuito como parámetro?

In [15]:
            semilla="1011"
           
            resultado=nlfsr(semilla, 500)

            resultado="".join(map(str, resultado.astype(int)))
            n=len(resultado)
            periods = [i for i in range(2,n//2+1) if resultado[:i]*(n//i)==resultado[:n - n % i]]

            if np.size(periods)==0:
                print("El periodo para la función planteada es", n)
            else:
                print("El periodo para la función planteada es", periods[0])

El periodo para la función planteada es 15


In [16]:
resultado

'11011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011001010000111011'

# La salida no es esa

In [17]:
def geffe(lfsr1, lfsr2, lfsr3, salida):

    # Transformo las salidas de los lfsr para poder operar
    # con ellas con la operaciones de bits inherentes al 
    # tipo int, paso de un array de numpy a un entero

    x1=int("".join(map(str, lfsr1.astype(int)))[::-1],2)
    x2=int("".join(map(str, lfsr2.astype(int)))[::-1],2)
    x3=int("".join(map(str, lfsr3.astype(int)))[::-1],2)

    # Combinamos las salidas de los LFSR's mediante
    # la función no lineal de la definicion y las funciones de
    # bits que tiene el tipo int

    # Al usar valores de L que son primos entre ellos 
    # tenemos que el periodo del resultado será de 
    # (2^L1-1)(2^L2-1)(2^L3-1)
    
    resultado=""
    for i in np.arange(salida):

        # Me quedo con el bit menos significativo
        x1i, x2i, x3i= x1 & 1, x2 & 1, x3 & 1
        
        # Desplazo a la derecha
        x1, x2, x3=x1>>1, x2>>1, x3>>1
        
        # Opero con el bit guardado
        nextValor=(x1i & x2i) ^ ((1 ^ x2i) & x3i)
        
        # Añado el resultado
        resultado=resultado+str(nextValor)

    # Devuelvo el string con la cadena
    return resultado


In [18]:
            # Generador de Geffe
            print("""El generador de Geffe con 3 lfsr, con L's 2, 3 y 5, y sus correspondientes polinomios primitivos obtiene:""")

            # Tomamos 3 valores de L que sean primos relativos entre
            # ellos, tomo como valores 3, 4, 5 y estos serán los grados
            # de los polinomios primitivos de los 3 LFSR, los listo
            # a continuación (no se representa el termino independiente)
            # además de las semillas de tamaño L y aleatorias
            p1, p2, p3= "11","101", "10010"

            # Semillas distintas
            s1, s2, s3= "10","100", "01100"

            # Guardamos el L para cada LFSR
            l1, l2, l3=len(p1), len(p2), len(p3)

            # Periodo máximo al hacerlo con valores de L
            # que son primos relativos
            periodo=((2**l1)-1)*((2**l2)-1)*((2**l3)-1)

            # Calculamos los 3 LFSR de periodo máximo
            lfsr1=lfsr(periodo, p1, s1)
            lfsr2=lfsr(periodo, p2, s2)
            lfsr3=lfsr(periodo, p3, s3)

            # Calculamos "periodo" elementos con geffe, y debemos obtener 
            # que el periodo es igual al número de la variable "periodo"
            cadenaGeffe=geffe(lfsr1, lfsr2, lfsr3, periodo)

            # Calculo del periodo
            # https://stackoverflow.com/questions/63181869/how-do-i-measure-the-periodicity-or-frequency-of-a-list-of-values
            
            n=len(cadenaGeffe)
            periods = [i for i in range(2,n//2+1) if cadenaGeffe[:i]*(n//i)==cadenaGeffe[:n - n % i]]

            if np.size(periods)==0:
                print("Tiene periodo", n, "es decir del tamaño del vector y por tanto periodo máximo")
            else:
                print("Tiene periodo minimo de", periods[0])

            # Cifro un m arbitrario(un número, un mensaje etc)
            m=32

            # Construyo una llave del mismo tamaño que m
            k=geffe(lfsr1, lfsr2, lfsr3, len(bin(m)[2:]))

            # Cifro m
            mCifrada=int(k,2) ^ m

            print("Cifro el valor",m,"obteniendo",mCifrada)

            # Descifro m
            mDescifrada=int(k,2) ^ mCifrada
            print("Y descifro",mCifrada,"obteniendo",mDescifrada)


El generador de Geffe con 3 lfsr, con L's 2, 3 y 5, y sus correspondientes polinomios primitivos obtiene:
Tiene periodo 651 es decir del tamaño del vector y por tanto periodo máximo
Cifro el valor 32 obteniendo 42
Y descifro 42 obteniendo 32


In [19]:
3*7*31

651

In [20]:
cadenaGeffe

'001010000011110111001101011010011110111011111011000011011111001101010011011111111001011010001011110011011101010000011110111001101011010011110111011111011000011011111001101010011011111111001011010001011110011011101010000011110111001101011010011110111011111011000011011111001101010011011111111001011010001011110011011101010000011110111001101011010011110111011111011000011011111001101010011011111111001011010001011110011011101010000011110111001101011010011110111011111011000011011111001101010011011111111001011010001011110011011101010000011110111001101011010011110111011111011000011011111001101010011011111111001011010001011110011011101010000011110111001'

# No implementas una función de cifrado y descifrado

In [71]:
def berlekampMassey(cadena):
    # Tamaño de la cadena
    n=np.size(cadena)

    # Inicialización
    polinomio, b, T = 1, 1, -1
    L, m, N, d = 0, -1, 0, -1

    while N<n:

        # Siguiente discrepancia
        cadenaInt=int("".join(map(str, cadena[-L:].astype(int))),2)
        d=sumaBitsEntero(cadena[N]+sumaBitsEntero(cadenaInt & polinomio))

        # Si la discrepancia es 1 hacemos lo siguiente
        if d==1:
            T=polinomio
            polinomio=polinomio + (b * (1<<(N-m)))

            if L<=N//2:
                L=N+1-L
                m=N
                b=T
        N+=1
    
    return L


In [72]:
cg = []+[int(a) for a in cadenaGeffe]

In [73]:
berlekampMassey(np.array(cg))

326

In [74]:
prueba=lfsr(10,"11","11")

In [75]:
pnp = np.array([int(a) for a in prueba])

In [76]:
pnp

array([1, 1, 0, 1, 1, 0, 1, 1, 0, 1])

In [77]:
berlekampMassey(pnp)

5

# Berlekamp-Massey parece no funcionar

In [78]:
prueba=lfsr(100,"10111","11111")

In [79]:
prueba

array([1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.])

In [80]:
pnp = np.array([int(a) for a in prueba])

In [81]:
pnp

array([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1])

In [82]:
berlekampMassey(pnp)

50

In [83]:
pnp

array([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1])